## Instalação de bibliotecas

In [164]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
!pip install transformers -q

In [3]:
import random
import torch
import torch.nn.functional as F
import numpy as np

Definição da semente para aleatoriedade, para fins de reprodução do treinamento.

In [4]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Download e processamento das bases (treinamento e teste)

In [5]:
!wget https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

--2023-03-10 22:48:20--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘msmarco_triples.train.tiny.tsv’

msmarco_triples.tra 100%[===================>]   7.70M  --.-KB/s    in 0.04s   

2023-03-10 22:48:20 (217 MB/s) - ‘msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]



In [6]:
import pandas as pd

In [122]:
df = pd.read_csv('msmarco_triples.train.tiny.tsv', sep='\t', header=None)

In [123]:
df.head()

,0,1,2
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [124]:
df_pos = pd.DataFrame()
df_neg = pd.DataFrame()

for index, row in df.iterrows():
  df_pos = df_pos.append({"query":row[0], "passage":row[1], "score":1}, ignore_index=True)
  df_neg = df_neg.append({"query":row[0], "passage":row[2], "score":0}, ignore_index=True)
    

In [125]:
df_pos

,query,passage,score
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,1.0
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,1.0
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,1.0
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,1.0
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...",1.0
...,...,...,...
10995,where is maramal in skyrim,I'm having a rather difficult time in Skyrim t...,1.0
10996,which is a basic unit of a sugar molecule,The basic unit of carbohydrate is a single sug...,1.0
10997,deloitte consulting or phone number,A: The phone number for Deloitte Consulting Ll...,1.0
10998,"how many hours a day to work to get lunch in ,a","In the United States, most states require a mi...",1.0


In [126]:
df_neg

,query,passage,score
0,is a little caffeine ok during pregnancy,It is generally safe for pregnant women to eat...,0.0
1,what fruit is native to australia,"The kola nut is the fruit of the kola tree, a ...",0.0
2,how large is the canadian military,The Canadian Physician Health Institute (CPHI)...,0.0
3,types of fruit trees,"The kola nut is the fruit of the kola tree, a ...",0.0
4,how many calories a day are lost breastfeeding,"However, you still need some niacin each day; ...",0.0
...,...,...,...
10995,where is maramal in skyrim,One type of character isnât necessarily bett...,0.0
10996,which is a basic unit of a sugar molecule,According to Merriam-Webster and the Online Et...,0.0
10997,deloitte consulting or phone number,Virtual phone numbers are convenient for both ...,0.0
10998,"how many hours a day to work to get lunch in ,a",1 Levaquin can be taken at any time of the day...,0.0


In [127]:
from sklearn.model_selection import train_test_split

X_train_pos = df_pos.drop("score", axis=1)
Y_train_pos = df_pos["score"]

X_train_pos, X_val_pos, Y_train_pos, Y_val_pos = train_test_split(X_train_pos, Y_train_pos, test_size=0.2, random_state=42)

X_train_neg = df_neg.drop("score", axis=1)
Y_train_neg = df_neg["score"]

X_train_neg, X_val_neg, Y_train_neg, Y_val_neg = train_test_split(X_train_neg, Y_train_neg, test_size=0.2, random_state=42)

In [128]:
X_train_pos

,query,passage
10735,"how far is cumberland md from washington, dc","Driving distance from Washington, DC to Cumber..."
5937,average cost to move and set up mobile home,"1 On average, a short mobile home move that is..."
7642,how big can siberian huskies get,male husky can get 23 feet long and females ca...
3328,how long can you hold someone's attention,"So much so, that as you approach the pulpit, t..."
8681,hcpcs code for medication management,"Inpatient telehealth pharmacologic management,..."
...,...,...
5734,how much did vanderbilt breakers mansion cost ...,"The mansion cost $11 million ($260,000,000 in ..."
5191,weather in dunedin,Dunedin Average Weather. 1 The highest monthl...
5390,meaning of the name brid,The name Brid is a Celtic baby name. In Celtic...
860,meaning of trap femboy,"From what I can see, trap as a noun meaning sp..."


In [129]:
Y_train_pos

10735    1.0
5937     1.0
7642     1.0
3328     1.0
8681     1.0
        ... 
5734     1.0
5191     1.0
5390     1.0
860      1.0
7270     1.0
Name: score, Length: 8800, dtype: float64

In [130]:
X_val_pos

,query,passage
107,where to get corso riggs,Background. Always cheerful and eternally opti...
5484,what rivers flow into the wabash river,The Wabash River rises south of Grand Lake abo...
6998,where is fiji islands,1 The Fiji Islands are an archipelago of over ...
3984,how long do nipple piercings take to heal,"Typically, nipple piercings take anywhere from..."
3111,what is a pole pocket banner,Pole pockets on banners are something that can...
...,...,...
5284,what are bacterial organisms,Taxonomy. Bacteria are members of the prokaryo...
3909,what county is park ridge nj,"Park Ridge, New Jersey. Park Ridge is a boroug..."
10549,what kind of tea gets rid of water retention,"To get rid of water retention, you must follow..."
5809,bordeaux meaning,Bordeaux is a port city on the Garonne River i...


In [131]:
Y_val_pos

107      1.0
5484     1.0
6998     1.0
3984     1.0
3111     1.0
        ... 
5284     1.0
3909     1.0
10549    1.0
5809     1.0
7571     1.0
Name: score, Length: 2200, dtype: float64

In [132]:
X_train_neg

,query,passage
10735,"how far is cumberland md from washington, dc",Averages for Washington DC in September. The w...
5937,average cost to move and set up mobile home,The average cost per square foot to build a ho...
7642,how big can siberian huskies get,If a normal tour of duty to Siberia is 12 mont...
3328,how long can you hold someone's attention,Can't take my eyes off of you. You'd be like h...
8681,hcpcs code for medication management,"In addition, Samsung created a Supplier Code o..."
...,...,...
5734,how much did vanderbilt breakers mansion cost ...,The average cost per square foot to build a ho...
5191,weather in dunedin,"Weather for Bradford, IL - Bradford, IL 61421 ..."
5390,meaning of the name brid,German Meaning: The name Jay is a German baby ...
860,meaning of trap femboy,Full Definition of DEATH TRAP. : a structure o...


In [133]:
Y_train_neg

10735    0.0
5937     0.0
7642     0.0
3328     0.0
8681     0.0
        ... 
5734     0.0
5191     0.0
5390     0.0
860      0.0
7270     0.0
Name: score, Length: 8800, dtype: float64

In [134]:
X_val_neg

,query,passage
107,where to get corso riggs,show more I have a male Cane Corso. I have rea...
5484,what rivers flow into the wabash river,"As such, the city's culture is also closely ti..."
6998,where is fiji islands,"On 24 July 2003, six member states of the Paci..."
3984,how long do nipple piercings take to heal,Fluid that leaks from one or both nipples is c...
3111,what is a pole pocket banner,The Geographic North Pole is located approxima...
...,...,...
5284,what are bacterial organisms,Cucumber bacterial wilt is transmitted by the ...
3909,what county is park ridge nj,It was a sleazy thing to do. I complained to A...
10549,what kind of tea gets rid of water retention,Hereâs the way you make a great cup of tulsi...
5809,bordeaux meaning,The capital of France is Paris. Largest Cities...


In [135]:
Y_val_neg

107      0.0
5484     0.0
6998     0.0
3984     0.0
3111     0.0
        ... 
5284     0.0
3909     0.0
10549    0.0
5809     0.0
7571     0.0
Name: score, Length: 2200, dtype: float64

In [136]:
X_train = pd.concat([X_train_pos, X_train_neg], axis=0, ignore_index=True)
Y_train = pd.concat([Y_train_pos, Y_train_neg], axis=0, ignore_index=True)
X_val = pd.concat([X_val_pos, X_val_neg], axis=0, ignore_index=True)
Y_val = pd.concat([Y_val_pos, Y_val_neg], axis=0, ignore_index=True)

In [137]:
X_train

,query,passage
0,"how far is cumberland md from washington, dc","Driving distance from Washington, DC to Cumber..."
1,average cost to move and set up mobile home,"1 On average, a short mobile home move that is..."
2,how big can siberian huskies get,male husky can get 23 feet long and females ca...
3,how long can you hold someone's attention,"So much so, that as you approach the pulpit, t..."
4,hcpcs code for medication management,"Inpatient telehealth pharmacologic management,..."
...,...,...
17595,how much did vanderbilt breakers mansion cost ...,The average cost per square foot to build a ho...
17596,weather in dunedin,"Weather for Bradford, IL - Bradford, IL 61421 ..."
17597,meaning of the name brid,German Meaning: The name Jay is a German baby ...
17598,meaning of trap femboy,Full Definition of DEATH TRAP. : a structure o...


In [138]:
Y_train

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
17595    0.0
17596    0.0
17597    0.0
17598    0.0
17599    0.0
Name: score, Length: 17600, dtype: float64

In [139]:
X_val

,query,passage
0,where to get corso riggs,Background. Always cheerful and eternally opti...
1,what rivers flow into the wabash river,The Wabash River rises south of Grand Lake abo...
2,where is fiji islands,1 The Fiji Islands are an archipelago of over ...
3,how long do nipple piercings take to heal,"Typically, nipple piercings take anywhere from..."
4,what is a pole pocket banner,Pole pockets on banners are something that can...
...,...,...
4395,what are bacterial organisms,Cucumber bacterial wilt is transmitted by the ...
4396,what county is park ridge nj,It was a sleazy thing to do. I complained to A...
4397,what kind of tea gets rid of water retention,Hereâs the way you make a great cup of tulsi...
4398,bordeaux meaning,The capital of France is Paris. Largest Cities...


In [140]:
Y_val

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
4395    0.0
4396    0.0
4397    0.0
4398    0.0
4399    0.0
Name: score, Length: 4400, dtype: float64

In [141]:
X_train

,query,passage
0,"how far is cumberland md from washington, dc","Driving distance from Washington, DC to Cumber..."
1,average cost to move and set up mobile home,"1 On average, a short mobile home move that is..."
2,how big can siberian huskies get,male husky can get 23 feet long and females ca...
3,how long can you hold someone's attention,"So much so, that as you approach the pulpit, t..."
4,hcpcs code for medication management,"Inpatient telehealth pharmacologic management,..."
...,...,...
17595,how much did vanderbilt breakers mansion cost ...,The average cost per square foot to build a ho...
17596,weather in dunedin,"Weather for Bradford, IL - Bradford, IL 61421 ..."
17597,meaning of the name brid,German Meaning: The name Jay is a German baby ...
17598,meaning of trap femboy,Full Definition of DEATH TRAP. : a structure o...


In [142]:
Y_train

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
17595    0.0
17596    0.0
17597    0.0
17598    0.0
17599    0.0
Name: score, Length: 17600, dtype: float64

In [143]:
X_val

,query,passage
0,where to get corso riggs,Background. Always cheerful and eternally opti...
1,what rivers flow into the wabash river,The Wabash River rises south of Grand Lake abo...
2,where is fiji islands,1 The Fiji Islands are an archipelago of over ...
3,how long do nipple piercings take to heal,"Typically, nipple piercings take anywhere from..."
4,what is a pole pocket banner,Pole pockets on banners are something that can...
...,...,...
4395,what are bacterial organisms,Cucumber bacterial wilt is transmitted by the ...
4396,what county is park ridge nj,It was a sleazy thing to do. I complained to A...
4397,what kind of tea gets rid of water retention,Hereâs the way you make a great cup of tulsi...
4398,bordeaux meaning,The capital of France is Paris. Largest Cities...


In [144]:
Y_val

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
4395    0.0
4396    0.0
4397    0.0
4398    0.0
4399    0.0
Name: score, Length: 4400, dtype: float64

## Finetuning

Preparação dos dados, com tokenização e construção do dataset e dataloaders.

In [145]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'microsoft/MiniLM-L12-H384-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [146]:
from statistics import mean, stdev

lengths = [len(tokens) for tokens in tokenizer(list(X_train["passage"])[:1_000])['input_ids']]
print(f'Mean length in tokens: {mean(lengths):0.2f}')
print(f'Stdev length in tokens: {stdev(lengths):0.2f}')

Mean length in tokens: 80.45
Stdev length in tokens: 31.20


In [147]:
train_queries = list(X_train["query"])
train_passages = list(X_train["passage"])
val_queries = list(X_val["query"])
val_passages = list(X_val["passage"])
max_length = 512
train_queries_tokenized = tokenizer(train_queries, max_length=max_length, truncation=True)
train_passages_tokenized = tokenizer(train_passages, max_length=max_length, truncation=True)
val_queries_tokenized = tokenizer(val_queries, max_length=max_length, truncation=True)
val_passages_tokenized = tokenizer(val_passages, max_length=max_length, truncation=True)

In [148]:
train_queries_tokenized[0]

Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [149]:
train_queries_tokenized[0].ids

[101, 2129, 2521, 2003, 12310, 9108, 2013, 2899, 1010, 5887, 102]

In [150]:
train_queries_tokenized[0].tokens

['[CLS]',
 'how',
 'far',
 'is',
 'cumberland',
 'md',
 'from',
 'washington',
 ',',
 'dc',
 '[SEP]']

In [151]:
len(train_queries_tokenized[0].tokens)

11

In [152]:
len(train_queries_tokenized['input_ids'][0])

11

In [153]:
train_queries_tokenized[0].attention_mask

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [154]:
len(train_queries_tokenized[0].attention_mask)

11

In [159]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, queries, passages, targets):
        self.queries = queries
        self.passages = passages
        self.targets = targets
    
    def __len__(self):
        return len(self.queries['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.queries['input_ids'][idx] + self.passages['input_ids'][idx],
            'attention_mask': self.queries['attention_mask'][idx] + self.passages['attention_mask'][idx],
            'labels': int(self.targets[idx]),
        }

dataset_train = Dataset(train_queries_tokenized, train_passages_tokenized, Y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0
assert type(dataset_train[2]['labels']) == int

In [160]:
dataset_valid = Dataset(val_queries_tokenized, val_passages_tokenized, Y_val)

In [161]:
from transformers import BatchEncoding

# Tokens do tipo "pad" para textos com tamanho inferior ao máximo suportado.
def collate_fn(batch):
  return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn)
#dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=False, collate_fn=collate_fn)

for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


### Training Loop

In [162]:
def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')

In [163]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
print('Parameters', model.num_parameters())

epochs = 5
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(dataloader_train)

num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in tqdm(range(epochs), desc='Epochs'):
    model.train()
    train_losses = []
    for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
        optimizer.zero_grad()
        outputs = model(**batch.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.cpu().item())

    print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.694; Valid accuracy: 0.500


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

...

Valid loss: 0.224; Valid accuracy: 0.934


Train:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.04


Valid:   0%|          | 0/138 [00:00<?, ?it/s]

Valid loss: 0.247; Valid accuracy: 0.937


In [167]:
MODELS_PATH = '/content/gdrive/MyDrive/Unicamp-aula-3'
model.save_pretrained(f'{MODELS_PATH}/models_ranker')
tokenizer.save_pretrained(f'{MODELS_PATH}/tokenizer_ranker')

('/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer_config.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/special_tokens_map.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/vocab.txt',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/added_tokens.json',
 '/content/gdrive/MyDrive/Unicamp-aula-3/tokenizer_ranker/tokenizer.json')